<a href="https://colab.research.google.com/github/bwsi-hadr/07-Satellite_Image_Processing_Using_Python/blob/master/07b_Integrating_raster_and_vector_GIS_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integrating raster and vector GIS data
We've now had experience working with both raster and vector GIS data individually. Now we'll look into how to integrate the two data types.



In [ ]:
# need to specify location of some certificates for rasterio
!export CURL_CA_BUNDLE=/etc/ssl/certs/ca-certificates.crt
try:
  import rasterio
  import rasterio.plot
  import rasterio.merge 
  import rasterio.mask
except:
  !pip install rasterio
  import rasterio
  import rasterio.plot
  import rasterio.merge
  import rasterio.mask
  
try:
  import rasterstats as rs
except:
  !pip install rasterstats  
  import rasterstats as rs
  
try:
  import pyproj
except:
  !pip install pyproj
  import pyproj
  
import networkx as nx
try:
  import osmnx as ox
except:
  # osmnx depends on the system package libspatialindex
  !apt install libspatialindex-dev
  !pip install osmnx
  import osmnx as ox

try: 
  import geopandas as gpd
except: 
  !pip install geopandas 
  import geopandas as gpd
  
try:
  import contextily as ctx 
except:
  # install dependencies for contextily
  !apt install libproj-dev proj-data proj-bin
  !apt install libgeos-dev
  !pip install cython
  !pip install cartopy
  # install contextily
  !pip install contextily==1.0rc1 --no-use-pep517 --no-cache-dir
  import contextily as ctx
  
import fiona
from shapely.geometry import Point, LineString, Polygon
  
import gdal
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

## Loading landsat


In [ ]:
print('Landsat on AWS:')
filepath = 'http://landsat-pds.s3.amazonaws.com/c1/L8/021/030/LC08_L1TP_021030_20180702_20180716_01_T1/LC08_L1TP_021030_20180702_20180716_01_T1_B2.TIF'
with rasterio.open(filepath) as src:
    print(src.profile)



## Getting topological raster data
We're going to get some elevation data from USGS. You can search for data products from USGS [here](https://viewer.nationalmap.gov/basic/#productGroupSearch). We are using elevation products (3DEP) at 1/3 arc-second (~10m) resolution. Each tile spans 1x1 degree.

The files are available as .zip of [Erdas Imagine .img](https://gdal.org/drivers/raster/hfa.html) filetypes. We'll be using the rasterio `zip+https://` [syntax](https://rasterio.readthedocs.io/en/stable/topics/datasets.html) to get the unzipped .img files from the url.

In [ ]:
# Get raster of boston-area elevations (takes a while)
tile1_url = 'zip+https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/13/IMG/USGS_NED_13_n43w072_IMG.zip!USGS_NED_13_n43w072_IMG.img'
tile2_url = 'zip+https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/13/IMG/USGS_NED_13_n43w071_IMG.zip!USGS_NED_13_n43w071_IMG.img'

# we'll write them to local files so we don't have to keep downloading
local_tile1_location = 'local_tile1.tif'
local_tile2_location = 'local_tile2.tif'

We'll write a local copy so we don't have to keep downloading

In [ ]:
def download_to_local(tile_url, local_tile_path):
  with rasterio.Env():
    with rasterio.open(tile_url) as src:
      profile = src.profile
      profile['dtype'] = 'float64'
      profile['indexes'] = 1
      print(profile)
      with rasterio.open(local_tile_path, 'w', **profile) as dst:
        dst.write_band(1,src.read(1).squeeze().astype(np.float64))

In [ ]:
download_to_local(tile1_url, local_tile1_location)

In [ ]:
download_to_local(tile2_url, local_tile2_location)

In [ ]:
# the files are really big, so we'll open them at half resolution each
with rasterio.open(local_tile1_location) as src:
  print(src.profile)
  tile1 = src.read(1, out_shape=(int(src.height/2), int(src.width/2)), 
                   resampling=rasterio.enums.Resampling.bilinear)
  tile1_transform = src.profile['transform']
  
with rasterio.open(local_tile2_location) as src:
  print(src.profile)
  tile2 = src.read(1, out_shape=(int(src.height/2), int(src.width/2)), 
                   resampling=rasterio.enums.Resampling.bilinear)
  tile2_transform = src.profile['transform']
  

In [ ]:
print(tile1.shape, tile2.shape)

In [ ]:
fig = plt.figure(figsize=[10,10])
ax1 = fig.add_subplot(2,1,1)
rasterio.plot.show(tile1, ax=ax1, transform=tile1_transform)
ax2 = fig.add_subplot(2,1,2)
rasterio.plot.show(tile2, ax=ax2, transform=tile2_transform)


The second part looks weird, right? That's because it contains `nodata` values, which are `-3.4028234663853e+38`. That makes the low end way too low. 

Fortunately, there's a masking function that lest us select all of the non-nodata pixels

In [ ]:
with rasterio.open(local_tile1_location) as src:
  tile1_mask = src.read_masks(1, out_shape=(int(src.height/2), int(src.width/2)), 
                   resampling=rasterio.enums.Resampling.bilinear)
  

with rasterio.open(local_tile2_location) as src:
  tile2_mask = src.read_masks(1, out_shape=(int(src.height/2), int(src.width/2)), 
                   resampling=rasterio.enums.Resampling.bilinear)
  


In [ ]:
# we use the np.ma.masked_where() function to mask out all the values equal to zero
fig = plt.figure(figsize=[10,10])
ax1 = fig.add_subplot(2,1,1)
rasterio.plot.show(np.ma.masked_where(tile1_mask==0, tile1), ax=ax1, transform=tile1_transform)
ax2 = fig.add_subplot(2,1,2)
rasterio.plot.show(np.ma.masked_where(tile2_mask==0, tile2), ax=ax2, transform=tile2_transform)

Let's combine the two tiles together into one large mosaic and save it as a geotiff

In [ ]:
combined_location = 'combined.tif'

In [ ]:
with rasterio.open(local_tile1_location, 'r') as src1:
  with rasterio.open(local_tile2_location, 'r') as src2:
    combined, out_transform = rasterio.merge.merge([src1, src2],
                                                   nodata=src2.profile['nodata'])
    combined_meta = src1.meta.copy()
    combined_meta.update({'driver':'GTiff',
                     'count': combined.shape[0],
                    'height': combined.shape[1],
                    'width': combined.shape[2],
                    'transform': out_transform,
                    'crs': src1.crs})
    with rasterio.open(combined_location, 'w', **combined_meta) as dst:
      dst.write(combined)


In [ ]:
with rasterio.open(combined_location, 'r') as src:
  print(src.meta)
  combined = src.read(1)
  combined_mask = src.read_masks(1)
  combined_transform = src.profile['transform']
  rasterio.plot.show(np.ma.masked_where(combined_mask==0, combined), 
                     transform=combined_transform)

In [ ]:
camb_name = "Cambridge, MA, USA"
graph_area_cambridge = ox.gdf_from_place(camb_name)
bos_name = "Boston, MA, USA"
graph_area_bos = ox.gdf_from_place(bos_name)
local_areas = graph_area_bos.append(graph_area_cambridge, ignore_index=True)
local_areas.to_crs(epsg=4269).to_file('local_areas.shp')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
p = rasterio.plot.show(np.ma.masked_where(combined_mask==0, combined), 
                       ax=ax, transform=combined_transform)
graph_area_cambridge.to_crs(epsg=4269).plot(ax=ax, facecolor='none', edgecolor='red')
graph_area_bos.to_crs(epsg=4269).plot(ax=ax, facecolor='none', edgecolor='magenta')
plt.show()


In [ ]:
rs.zonal_stats('local_areas.shp','combined.tif',stats='mean max')